In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

According to creditcards.com, there was over £300m in fraudulent credit card transactions in the UK in the first half of 2016, with banks preventing over £470m of fraud in the same period. The data shows that credit card fraud is rising, so there is an urgent need to continue to develop new, and improve current, fraud detection methods.

Using this dataset, we will use machine learning to develop a model that attempts to predict whether or not a transaction is fraudlent. To preserve anonymity, these data have been transformed using principal components analysis.

In [ ]:
# load packages
library(readr)
library(dplyr)
library(randomForest)
library(ggplot2)
library(Hmisc)
library(party)

In [ ]:
# set random seed for model reproducibility
set.seed(1234)

In [ ]:
# import data
creditData <- read_csv("../input/creditcardfraud/creditcard.csv")

In [ ]:
# look at the data
glimpse(creditData)

In [ ]:
head(creditData)

In [ ]:
# make Class a factor
creditData$Class <- factor(creditData$Class)

Testing and Training

In [ ]:
train <- creditData[1:150000, ]
test <- creditData[150001:284807, ]

In [ ]:
train %>%
  select(Class) %>%
  group_by(Class) %>%
  summarise(count = n()) %>%
  glimpse

In [ ]:
test %>%
  select(Class) %>%
  group_by(Class) %>%
  summarise(count = n()) %>%
  glimpse

As we can see, fraudulent transactions are a very small proportion of our dataset, we could build what would appear to be a highly accurate model just by always saying that every transaction was not fraudulent. While we would be right over 99% of the time, that would cost consumers and the industry over £500m per year, so wouldn't be a useful model.

In [ ]:
# build random forest model using every variable
rfModel <- randomForest(Class ~ . , data = train)

In [ ]:
test$predicted <- predict(rfModel, test)

So we've built a random forest model using all the available variables and used it to predict whether or not a transaction is fraudulent on our test test. As the data is very imbalanced, we would expect our accuracy to be very high, even if our model just always guessed 'not fraudulent'.

In [ ]:
library(caret)
confusionMatrix(test$Class, test$predicted)

Looking at the output above, we can indeed see that we have very high accuracy (99.94%) as we expected. Going back to our test set class counts, we can see that we had 134,608 legitimate transactions, and 199 fraudulent transactions. Had we said that every transaction was fraudulent, we would have got 199 wrong.

In [ ]:
library(MLmetrics)

In [ ]:
F1_all <- F1_Score(test$Class, test$predicted)
F1_all

In [ ]:
options(repr.plot.width=5, repr.plot.height=4)
varImpPlot(rfModel,
          sort = T,
           n.var=10,
           main="Top 10 Most Important Variables")

In [ ]:
rfModelTrim1 <- randomForest(Class ~  V17, 
                            data = train)

test$predictedTrim1 <- predict(rfModelTrim1, test)

F1_1 <- F1_Score(test$Class, test$predictedTrim1)
F1_1

In [ ]:
rfModelTrim2 <- randomForest(Class ~  V17 + V12, 
                            data = train)

test$predictedTrim2 <- predict(rfModelTrim2, test)

F1_2 <- F1_Score(test$Class, test$predictedTrim2)
F1_2

In [ ]:
rfModelTrim3 <- randomForest(Class ~  V17 + V12 + V14, 
                            data = train)

test$predictedTrim3 <- predict(rfModelTrim3, test)

F1_3 <- F1_Score(test$Class, test$predictedTrim3)
F1_3

In [ ]:
# four variables
rfModelTrim4 <- randomForest(Class ~  V17 + V12 + V14 + V10, 
                            data = train)

test$predictedTrim4 <- predict(rfModelTrim4, test)

F1_4 <- F1_Score(test$Class, test$predictedTrim4)
F1_4

In [ ]:
# five variables
rfModelTrim5 <- randomForest(Class ~  V17 + V12 + V14 + V10 + V16, 
                            data = train)

test$predictedTrim5 <- predict(rfModelTrim5, test)

F1_5 <- F1_Score(test$Class, test$predictedTrim5)
F1_5

In [ ]:
# ten variables
rfModelTrim10 <- randomForest(Class ~  V17 + V12 + V14 + V10 + V16 
                              + V11 + V9 + V4 + V18 + V26, 
                            data = train)

test$predictedTrim10 <- predict(rfModelTrim10, test)

F1_10 <- F1_Score(test$Class, test$predictedTrim10)
F1_10

With those scores calculated, let's go ahead and plot those out:

In [ ]:
# build dataframe of number of variables and scores
numVariables <- c(1,2,3,4,5,10,17)
F1_Score <- c(F1_1, F1_2, F1_3, F1_4, F1_5, F1_10, F1_all)
variablePerf <- data.frame(numVariables, F1_Score)

In [ ]:
# plot score performance against number of variables
options(repr.plot.width=4, repr.plot.height=3)
ggplot(variablePerf, aes(numVariables, F1_Score)) + geom_point() + labs(x = "Number of Variables", y = "F1 Score", title = "F1 Score Performance")

In [ ]:
rf10 = randomForest(Class ~  V17 + V12 + V14 + V10 + V16 
                              + V11 + V9 + V4 + V18 + V26,  
                   ntree = 1000,
                   data = train)

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
plot(rf10)

Plotting our 10-variable model shows that there is not much additional performance gained after what looks like about 50 trees, but let's zoom in on that region to make sure:

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
plot(rf10, xlim=c(0,100))

## Summing up
We have used a random forest method to predict whether or not a credit card transaction is fraudulent or not, and built a model that offers a useful uplift over the no information rate.

This basic model provides a starting point for contintinuing to tune the model to seek additional improvements. Although we are dealing with changes in accuracy at the fourth decimal place, these very slight changes in accuracy need to be considered with respect to the volume of credit card transactions that take place every year. Even with only a slight fraction of these being fraudulent, the sheer volume of transactions mean that even very slight improvements in model performance may result in significant reductions in credit card fraud.